In [ ]:
import sys, os
import netCDF4 as nc
sys.path.append('../src/')
from Biologging_Toolkit.wrapper import Wrapper
from Biologging_Toolkit.utils import *

In [ ]:
depid = 'ml17_280a'
path = '/home6/grosmaan/Documents/data'
sens_path = os.path.join(path, 'CTD', depid, depid+'sens5.nc')
raw_path = os.path.join(path, 'individus', depid, 'raw')

In [ ]:
inst = Wrapper(depid, 
            path = path 
              )

inst.dt = 5   # Timestep in seconds of final data structure

In [ ]:
inst.ds

### Create reference time data

Enter either a sens5 path containing existing time data or a time vector in POSIX timestamps (UTC)
In both cases, the data will be resampled to match inst.dt

Change overwrite to True if time data already exists and you wish to overwrite it.

In [ ]:
inst.create_time(time_path = sens_path, overwrite = False)

In [ ]:
inst.create_time(time_data = [151878955, 154324489], overwrite = False)

In [ ]:
inst.ds['time'][:]

### Load GPS data

Enter time, latitude and longitude data (all the same size) and it will be resampled to match reference time data

Change overwrite to True if GPS data already exists and you wish to overwrite it.

In [ ]:
trk_path = os.path.join('/run/media/grosmaan/LaCie/individus_brut/CTD', depid, depid+'trk.nc')
trk = nc.Dataset(trk_path)

In [ ]:
time, lat, lon = trk['POS'][:].data
time =  time + get_start_time_sens(trk.dephist_deploy_datetime_start)

In [ ]:
inst.create_gps(time_data=time, lat_data=lat, lon_data=lon, overwrite = True)

In [ ]:
inst.ds['lat'][:], inst.ds['lon'][:]

### Add any other data

You can now add any other data to structure, here we add depth as an example

In [ ]:
var_name = 'depth'
sens = nc.Dataset(sens_path)
var_data = sens['P'][:].data

length = len(sens['P'][:])
ds_sr = sens['P'].sampling_rate
time_data = get_start_time_sens(sens.dephist_device_datetime_start) + np.arange(0, length/ds_sr, np.round(1/ds_sr,2))

In [ ]:
inst.create_variable(var_name, var_data, time_data, overwrite = False)

In [ ]:
inst.ds.close()